In [3]:
import pandas as pd
import sys
sys.path.append('../')

from ais_trajectory_simplification.simplification.oepp_simplificator import OEPPSimplificator
from ais_trajectory_simplification.simplification.dp_simplificator import DPSimplificator
from ais_trajectory_simplification.simplification.downsampling_simplificator import DownsamplingSimplificator
from ais_trajectory_simplification.simplification.tdtr_simplificator import TDTRSimplificator


In [4]:
ship_pdf = pd.read_csv('../data/processed/cleaned.csv')
ship_pdf

,position_timestamp,mmsi,latitude,longitude,prev_pos_latitude,prev_pos_longitude,bearing_since_prev_pos_deg,distance_since_prev_pos_m,prev_pos_position_timestamp,time_since_prev_pos_s,speed_since_prev_pos_kn,prev_speed_since_prev_pos_kn,acceleration_kn_s,speed_reference_kn,no_of_cleaned_positions_since_prev_pos
0,2023-01-03 00:01:46,220614000,57.6671,11.8045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,0
1,2023-01-03 00:10:46,220614000,57.6670,11.8046,57.6671,11.8045,151.814460,12.635450,2023-01-03 00:01:46,540.0,0.045484,NaN,NaN,20,0
2,2023-01-03 00:22:46,220614000,57.6668,11.8048,57.6670,11.8046,151.814319,25.270922,2023-01-03 00:10:46,720.0,0.068226,0.045484,0.000032,20,0
3,2023-01-03 00:30:04,220614000,57.6667,11.8051,57.6668,11.8048,121.883010,21.085611,2023-01-03 00:22:46,438.0,0.093578,0.068226,0.000058,20,0
4,2023-01-03 00:40:47,220614000,57.6664,11.8051,57.6667,11.8051,180.000000,33.411526,2023-01-03 00:30:04,643.0,0.101006,0.093578,0.000012,20,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-01-03 23:20:02,220614000,57.1041,11.9381,57.1293,11.9220,160.827789,2970.990064,2023-01-03 23:10:01,601.0,9.609222,9.606506,0.000005,20,0
119,2023-01-03 23:30:01,220614000,57.0788,11.9530,57.1041,11.9381,162.217710,2958.703104,2023-01-03 23:20:02,599.0,9.601434,9.609222,-0.000013,20,0
120,2023-01-03 23:40:01,220614000,57.0534,11.9673,57.0788,11.9530,162.943626,2958.609644,2023-01-03 23:30:01,600.0,9.585128,9.601434,-0.000027,20,0
121,2023-01-03 23:50:02,220614000,57.0282,11.9813,57.0534,11.9673,163.145280,2932.156711,2023-01-03 23:40:01,601.0,9.483622,9.585128,-0.000169,20,0


In [ ]:
import time

def generate_simplify_trajectory_oepp(stop_max_distance_m, max_heading_deviation_deg, max_speed_deviation_kn):
    def simplify_trajectory(pdf: pd.DataFrame):
        simplificator = OEPPSimplificator(pdf)
        segments_pdf = simplificator.simplify_trajectory(stop_max_distance_m, max_heading_deviation_deg, max_speed_deviation_kn)
        return segments_pdf
    return simplify_trajectory

results = []
for stop_m in [100]:
    for max_heading_deviation_deg in [8]:
        for max_speed_deviation_kn in [3]:
            start_time = time.time()
            result_pdf = ship_pdf.groupby('mmsi').apply(generate_simplify_trajectory_oepp(stop_m, max_heading_deviation_deg, max_speed_deviation_kn))
            execution_time = time.time() - start_time
            result_pdf['params'] = f"{stop_m}_{max_heading_deviation_deg}_{max_speed_deviation_kn}"
            result_pdf['type'] = 'O-EPP'
            result_pdf['execution_time'] = execution_time
            results.append(result_pdf.copy())

pd.concat(results).reset_index(drop=True).to_csv('../data/processed/O-EPP.csv')


In [5]:
import time

def generate_simplify_trajectory_tdtr(epsilon_m):
    def simplify_trajectory(pdf: pd.DataFrame):
        simplificator = TDTRSimplificator(pdf)
        dp_pdf = simplificator.simplify_trajectory(epsilon_m)
        return dp_pdf
    return simplify_trajectory


results = []
for epsilon_m in [100]:
    start_time = time.time()
    result_pdf = ship_pdf.groupby('mmsi').apply(generate_simplify_trajectory_tdtr(epsilon_m))
    execution_time = time.time() - start_time
    result_pdf['type'] = 'Douglas-Peucker'
    result_pdf['params'] = f"{epsilon_m}"
    result_pdf['execution_time'] = execution_time
    results.append(result_pdf.copy())

pd.concat(results).reset_index(drop=True).to_csv('../data/processed/tdtr.csv')


In [ ]:
import time

def generate_simplify_trajectory_tdtr(epsilon_m):
    def simplify_trajectory(pdf: pd.DataFrame):
        simplificator = DPSimplificator(pdf)
        dp_pdf = simplificator.simplify_trajectory(epsilon_m)
        return dp_pdf
    return simplify_trajectory


results = []
for epsilon_m in [100]:
    start_time = time.time()
    result_pdf = ship_pdf.groupby('mmsi').apply(generate_simplify_trajectory_tdtr(epsilon_m))
    execution_time = time.time() - start_time
    result_pdf['type'] = 'Douglas-Peucker'
    result_pdf['params'] = f"{epsilon_m}"
    result_pdf['execution_time'] = execution_time
    results.append(result_pdf.copy())

pd.concat(results).reset_index(drop=True).to_csv('../data/processed/dp.csv')


In [ ]:
import time

def generate_simplify_trajectory_downsampling(downsampling_time_s):
    def simplify_trajectory(pdf: pd.DataFrame):
        downsampling_simplificator = DownsamplingSimplificator(pdf)
        downsampling_pdf = downsampling_simplificator.simplify_trajectory(downsampling_time_s)
        return downsampling_pdf
    return simplify_trajectory


results = []
for downsampling_sec in [60 * 60 * 0.5]:
    start_time = time.time()
    result_pdf = ship_pdf.groupby('mmsi').apply(generate_simplify_trajectory_downsampling(downsampling_sec))
    execution_time = time.time() - start_time
    result_pdf['type'] = 'Down-sampling'
    result_pdf['params'] = f"{downsampling_sec}"
    result_pdf['execution_time'] = execution_time
    results.append(result_pdf.copy())

pd.concat(results).reset_index(drop=True).to_csv('../data/processed/downsampling.csv')
